from v1 to v2 real function to implement in prepocess pipeline <br>
from v2 to v3 solve the header pb n computeStructural Image <br>
from v3 to v4 to find why the results are different <br>
v6 prendre en compte slope et intercept 

array_header.get_slope_inter()

In [7]:
img1 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii'
img2 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c2t0009_t1_s03.nii'
img3 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/mt0009_t1_s03.nii'


In [8]:
def computeStructuralImage(img1,img2,img3):
    ''' 
    takes 3 images arguments: grey matter, white matter, and bias corrected T1 img.
    compute a binary mask with gm and wm thresholded at 0.2 combined with bias correctd img.
    used to realigne properly the EPI imgs 
    different version to be in datatype INT16 unsigned
    '''

    import numpy as np     
    import nibabel as nib     
    import os 
    
    i1=nib.load(img1)         
    i1array=np.asarray(i1.dataobj).copy() # Avoid caching the proxy image
    
    i2=nib.load(img2)         
    i2array=np.asarray(i2.dataobj).copy()
    hdr2 = i2.header
    
    i3=nib.load(img3)         
    i3array=np.asarray(i3.dataobj).copy()
    hdr3 = i3.header
    
    print(hdr2.get_data_dtype())
    print(hdr3.get_data_dtype())
    
    gi = i1array + i2array
    # threshold image gm + wm at 0.2
    gi[(10 * gi )< 2] = 0
    # binary mask the resulting image
    gi[gi > 0] = 1
    
    struct_image = i3
    
    hdr_struct = struct_image.header
    print(hdr_struct.get_data_dtype())
    
    print"apres"
        
    print(hdr_struct.get_data_dtype())
    
    # apply the binary mask to the bias corrected T1
    struct_image_array = np.multiply(gi, i3array)
   
    hdr3.set_data_dtype(np.int16)
    
    # crate the resulting image
    struct_image = nib.Nifti1Image(struct_image_array.astype(np.int16), i3.affine, i3.header)
    
    nib.save(struct_image, 'struct_image.nii')
    
    result = nib.load('struct_image.nii')
    hdr_result = result.header
    
    print(hdr_result.get_data_dtype())
    
    return 'struct_image.nii'
    
    

img = nb.load(filename)
data = img.get_data()
if data.max() < 2**16 and data.min() >= 0:
    nb.Nifti1Image(data.astype(np.int16), img.affine, img.header).to_filename(fileout)
else:
    raise ValueError('data range cannot be accommodated in 16-bit format')
    
np.asarray(a, dtype=np.float32)

In [9]:
%cd /homes_unix/hirsch/_goldstd/goldstarResults

/homes_unix/hirsch/_goldstd/goldstarResults


In [10]:
%pwd

u'/homes_unix/hirsch/_goldstd/goldstarResults'

In [11]:
img4 = computeStructuralImage(img1,img2,img3)

uint8
float32
float32
apres
float32
int16


In [30]:
!fslhd /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/dmt0009_t1_s02.nii > /homes_unix/hirsch/_goldstd/goldstarResults/imgcalc1.txt
!fslhd struct_image.nii > /homes_unix/hirsch/_goldstd/goldstarResults/imgcalc2.txt
!diff imgcalc1.txt imgcalc2.txt

1c1
< filename       /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/dmt0009_t1_s02.nii
---
> filename       struct_image.nii
29c29
< scl_slope      0.044728
---
> scl_slope      1.000000
66c66
< descrip        spm - algebra
---
> descrip        Bias corrected


## essai transposition pour avoir les voxels de gm et voir les différences dans les 2 calculs

In [19]:
import numpy as np     
import nibabel as nib     
import os
# on regarde le grey matter
img1 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii'
i1=nib.load(img1)         
i1array=np.asarray(i1.dataobj).copy() 
i1array[(10 * i1array )< 2] = 0
    # binary mask the resulting image
i1array[i1array > 0] = 1
    

In [20]:
np.transpose(np.nonzero(i1array))

array([[ 24,  98, 114],
       [ 24,  98, 115],
       [ 24,  98, 116],
       ..., 
       [158, 111, 116],
       [158, 111, 117],
       [158, 111, 118]])

In [36]:
print r1array[24,  98, 114]
print r2array[24,  98, 114]

298.333720602
298


In [37]:
print r1array[158, 111, 116]
print r2array[158, 111, 116]

360.236849435
360


In [42]:
# x[np.where( x > 3.0 )] 
diff_res=r2array[np.where(( r1array-r2array) > 1.03 )]
print np.count_nonzero(diff_res)
# pas bon

0


In [34]:
diff_res = r1array - r2array
print np.count_nonzero(diff_res)

1166481


In [33]:
diff_res.shape

(180, 256, 256)

In [39]:
180*256*256 - 1166481

10629999

In [40]:
np.amax(diff_res) 

1.0222499743103981

In [44]:
print np.amin(r1array) 
print np.amax(r1array) 

0.0
1465.59235727


In [35]:
np.transpose(np.nonzero(diff_res))

array([[ 24,  98, 114],
       [ 24,  98, 115],
       [ 24,  98, 116],
       ..., 
       [158, 111, 116],
       [158, 111, 117],
       [158, 111, 118]])

In [16]:
res1 ='/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/dmt0009_t1_s02.nii'
res2= 'struct_image.nii'

In [18]:
import numpy as np     
import nibabel as nib     
import os 
r1=nib.load(res1)         
r1array=np.asarray(r1.dataobj).copy() # Avoid caching the proxy image
    
r2=nib.load(res2)         
r2array=np.asarray(r2.dataobj).copy()

In [94]:
np.array_equal(i1array, i2array)

False

In [26]:
#np.allclose(r1array, r2array, rtol=1e-01)
np.allclose(r1array, r2array, rtol=1)

True

In [13]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_results/structural

/homes_unix/hirsch/_pypipe/datadir/data_results/structural


In [29]:
# test
img1 = 'c1t0009_t1_s03.nii'
img2 = 'c2t0009_t1_s03.nii'
img3 = 'mt0009_t1_s03.nii'
result = computeStructuralImage(img1,img2,img3)

In [ ]:
i1=nb.load('c1t0009_t1_s03.nii')         
i1array=np.asarray(i1.dataobj).copy() # Avoid caching the proxy image, thanks G. V. !!

In [ ]:
i2=nb.load('c1t0009_t1_s03.nii')         
i2array=np.asarray(i2.dataobj).copy() 

In [ ]:
i3=nb.load('BiasField_t0009_t1_s03.nii')         
i3array=np.asarray(i3.dataobj).copy() 

In [ ]:
gi = i1array + i2array

In [ ]:
gi[90,:, 128]

In [ ]:
# threshold image wm + csf
# a[a > 10] = 0
gi[gi < 0.2] = 0

In [ ]:
# mask the resulting image
gi[gi > 0.0] = 1

In [ ]:
gi[90,:, 128]

In [ ]:
# numpy.multiply(x1, x2[, out])
struct_image_array = np.multiply(gi, i3array)

In [ ]:
wm_csf = gi[gi > 0.2]

In [ ]:
180*256*256

In [ ]:
gi[90,128, 128]

In [ ]:
i3array[90,128, 128]

In [ ]:
i3array[90,0, 128]

In [ ]:
struct_image_array[90,128, 128]

In [ ]:
struct_image_array[90,0, 128]

In [ ]:
dims= i1array.shape
print dims

In [ ]:
i1.header.get_data_shape()

In [ ]:
i2=nib.load('c1t0009_t1_s03.nii')

In [ ]:
i3=nib.load('c2t0009_t1_s03.nii')

In [ ]:
data2 = i2.get_data()

In [ ]:
data2.__class__.__name__

In [ ]:
data3 = i3.get_data()

In [ ]:
data = data2 + data3

In [ ]:
data_threashold = data[data>0.2]

In [ ]:
data_threashold

In [ ]:
data_threashold[10]

In [ ]:
data_threashold.head()

In [ ]:
len(data_threashold)

In [ ]:
nib.load()?